We will clean the csv file which has historical data.

KD line normalization
Z(1.5): 𝐾𝑡−1 < 𝐷𝑡−1,𝐷𝑡 < 𝐾𝑡, and (20 >= 𝐾𝑡−2 and 20 >=𝐾𝑡−1 and 25 <= 𝐾𝑡)
𝐴(1): 𝐾𝑡−1 < 𝐷𝑡−1,𝐷𝑡 < 𝐾𝑡, and (20 < 𝐾𝑡−2 < 80 or 20 < 𝐾𝑡−1 < 80 or 20 < 𝐾𝑡 < 80)
𝐵(-1.5): 𝐾𝑡−1 > 𝐷𝑡−1,𝐷𝑡 > 𝐾𝑡 and (𝐾𝑡−2 >= 80 and 𝐾𝑡−1 >= 80 and 𝐾𝑡 <= 75)
𝐵(-1): 𝐾𝑡−1 > 𝐷𝑡−1,𝐷𝑡 > 𝐾𝑡 and (20 < 𝐾𝑡−2 < 80 or 20 < 𝐾𝑡−1 < 80 or 20 < 𝐾𝑡 < 80)
𝐶(0): Otherwise

MACD normalization:
𝐴(1):𝐷𝐼𝐹𝑡−1 < 9𝑀𝐴𝐶𝐷𝑡−1 𝑎𝑛𝑑 9𝑀𝐴𝐶𝐷𝑡 < 𝐷𝐼𝐹𝑡
𝐵(-1): 𝐷𝐼𝐹𝑡−1 > 9𝑀𝐴𝐶𝐷𝑡−1 𝑎𝑛𝑑 9𝑀𝐴𝐶𝐷𝑡 > 𝐷𝐼𝐹𝑡
𝐶(0): Otherwise

Label Design 
a) Daily
1. Trend up: 𝐶𝑙𝑜𝑠𝑒𝑡+1 -𝐶𝑙𝑜𝑠𝑒𝑡 > 𝐶𝑙𝑜𝑠𝑒𝑡 × α
2. Flat trend: 𝐶𝑙𝑜𝑠𝑒𝑡 × α > 𝐶𝑙𝑜𝑠𝑒𝑡+1 -𝐶𝑙𝑜𝑠𝑒𝑡 >−𝐶𝑙𝑜𝑠𝑒𝑡 × α
3. Trend down: 𝐶𝑙𝑜𝑠𝑒𝑡+1 -𝐶𝑙𝑜𝑠𝑒𝑡 < −𝐶𝑙𝑜𝑠𝑒𝑡 × α
b) Short Term (use moving average)

NOTE:
1) for all 3 column of macd line ta have values, 33 period needed
2) for all 2 column of stochastic kd line ta have values, 16 period needed

In [ ]:
import pandas as pd
import numpy as np
import logging

#NOTES: We are currenlty assuming that all values are present in the excel file:
#the data is continous. TO ADD: check for discontnous data and find a replacement strategy.


class ETL:
    def __init__(self):
        print('created')

    def Load_Clean_AddTI_Data(self,input_file_path,interval='1d'):
        try:
            df= pd.read_csv(input_file_path,header=1) # second row is column names
            col_list = ['unix_start_datetime','start_date','symbol','open','high','low','close','base_volume','quote_volume','num_trades']
            df.columns = col_list
            df["interval"]= interval
            df.dropna(subset=['symbol','unix_start_datetime','open','close','high','low','base_volume'], inplace=True)
            try:
            #will contain two new columns, STOCH_14_3K and STOCH_14_3D
                df.ta.stoch(high='high', low='low', close='close', k=14, d=3, fillna=True, append=True)
            except Exception as e:
                logging.ERROR('Failed to add technical indicator data: \n' + str(e))
            return df.copy()
        except Exception as e:
            logging.ERROR('Failed to load and clean data: \n' + str(e))


In [34]:
for i in range(0,30):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [35]:
import pandas as pd
import pandas_ta as ta
import numpy as np
df= pd.read_csv('./historical_data/Binance_ETHUSDT_d.csv',header=1) # second row is column names
col_list = ['unix_start_datetime','start_date','symbol','open','high','low','close','base_volume','quote_volume','num_trades']
df.columns = col_list
df.ta.stoch(high='high', low='low', close='close', k=14, d=3, fillna=True, append=True)
# df = df[["STOCHk_14_3_3","STOCHd_14_3_3"]].replace(True, np.nan)

# as we get Nan values in the first 16 rows, we drop these values as we have ample data
# display(df[["STOCHk_14_3_3","STOCHd_14_3_3"]].head(20))

#adds technical indicators
#adds 2 columns "STOCHk_14_3_3","STOCHd_14_3_3"
df.ta.stoch(high='high', low='low', close='close', k=14, d=3, fillna=True, append=True)
MACD = ta.macd(df['close'])
df['MACD']=MACD['MACD_12_26_9']
df['MACDs']=MACD['MACDs_12_26_9']
#rename the columns
df= df.rename(columns={"STOCHk_14_3_3":"STOCHk","STOCHd_14_3_3":"STOCHd"})

#Create Labels: 2 types of label, a) short term b) daily
window_size=5
ma_minus = df['Price'].rolling(window=window_size).mean()
ma_plus = ma_minus.shift(-(window_size-1))
alpha = 0.005
n=5
resultA = []
resultB = []

for i in range(ma_plus.shape[0]):
    #NOTE: bull, bear flat or buy sell hold
    #TODO: what
    resultA[i]= "up" if (ma_plus[i]-ma_minus[i]) > (ma_minus[i]*alpha) else "down" if (ma_plus[i]-ma_minus[i]) < (-ma_minus[i]*alpha) else "flat"
    if(i==(ma_plus.shape[0] -1)):
        resultB[i]='flat'
    else:
        resultB[i]= "up" if (df.loc[i+1,"close"]- df.loc[i,"close"]) > (df.loc[i,"close"]*alpha) else "down" if (df.loc[i+1,"close"]-  df.loc[i,"close"]) < (-df.loc[i,"close"]*alpha) else "flat"

#Note: Dropping N/A values
df.drop([range(33)])

#Normalize data
for index, row in df.iloc[2:].iterrows():
    # KD line normalization
    if(df['STOCHk',index - 1] < df['STOCHd',index - 1] and row['STOCHk'] < row['STOCHd']):
        if( 20>= df['STOCHk',index - 2] and 20>= df['STOCHk',index - 1] and 25 <= row['STOCHk']):
            df['ti_STOCHkd',index]= 1.5
        elif( 20< df['STOCHk',index - 2]<80 or 20< df['STOCHk',index - 1]<80 or 20<row['STOCHk']<80):
            df['ti_STOCHkd',index]= 1
    elif(df['STOCHk',index - 1] > df['STOCHd',index - 1] and row['STOCHk'] > row['STOCHd']):
        if( 80 <= df['STOCHk',index - 2] and 80 <= df['STOCHk',index - 1] and 75 >= row['STOCHk']):
            df['ti_STOCHkd',index]= -1.5
        elif( 20 < df['STOCHk',index - 2]< 80 or 20 < df['STOCHk',index - 1]< 80 or 20 < row['STOCHk']< 80):
            df['ti_STOCHkd',index]= -1
    else:
        df['ti_STOCHkd',index]= 0

    #MACD Norlmalization
    if(df['MACD',index - 1] < df['MACDs',index - 1] and row['MACDs'] < row['MACD']):
        df['ti_MACD',index] = 1
    elif(df['MACD',index - 1] > df['MACDs',index - 1] and row['MACDs'] > row['MACD']):
        df['ti_MACD',index] = -1
    else:
        df['ti_MACD',index] = 0

# KD line normalization
# Z(1.5): 𝐾𝑡−1 < 𝐷𝑡−1,𝐷𝑡 < 𝐾𝑡, and (20 >= 𝐾𝑡−2 and 20 >= 𝐾𝑡−1 and 25 <= 𝐾𝑡)
# 𝐴(1): 𝐾𝑡−1 < 𝐷𝑡−1,𝐷𝑡 < 𝐾𝑡, and (20 < 𝐾𝑡−2 < 80 or 20 < 𝐾𝑡−1 < 80 or 20 < 𝐾𝑡 < 80)
# 𝐵(-1.5): 𝐾𝑡−1 > 𝐷𝑡−1,𝐷𝑡 > 𝐾𝑡 and (𝐾𝑡−2 >= 80 and 𝐾𝑡−1 >= 80 and 𝐾𝑡 <= 75)
# C(-1): 𝐾𝑡−1 > 𝐷𝑡−1,𝐷𝑡 > 𝐾𝑡 and (20 < 𝐾𝑡−2 < 80 or 20 < 𝐾𝑡−1 < 80 or 20 < 𝐾𝑡 < 80)
# D(0): Otherwise

# MACD normalization:
# 𝐴(1):𝐷𝐼𝐹𝑡−1 < 9𝑀𝐴𝐶𝐷𝑡−1 𝑎𝑛𝑑 9𝑀𝐴𝐶𝐷𝑡 < 𝐷𝐼𝐹𝑡
# 𝐵(-1): 𝐷𝐼𝐹𝑡−1 > 9𝑀𝐴𝐶𝐷𝑡−1 𝑎𝑛𝑑 9𝑀𝐴𝐶𝐷𝑡 > 𝐷𝐼𝐹𝑡
# 𝐶(0): Otherwise


SyntaxError: invalid syntax (3493086906.py, line 33)

In [14]:
df = pd.DataFrame({'Price': [10, 12, 8, 14, 16, 11, 9, 13, 15, 17]})
print(df.shift(5))
print(df.shift(-5))

   Price
0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5   10.0
6   12.0
7    8.0
8   14.0
9   16.0
   Price
0   11.0
1    9.0
2   13.0
3   15.0
4   17.0
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN


In [24]:
import pandas as pd

df = pd.DataFrame({'Price': [10, 12, 8, 14, 16, 11, 9, 13, 15, 17]})

# print(df)
window_size=5
ma_minus = df['Price'].rolling(window=window_size).mean()
ma_plus = ma_minus.shift(-(window_size-1))

print(ma_plus[0]-ma_minus[4])


0.0
<class 'pandas.core.series.Series'>


SyntaxError: unexpected EOF while parsing (1907080176.py, line 3)

In [5]:
df.shift(-2)

,A,B,C,D
0,3.0,30.0,33.0,51.0
1,4.0,40.0,44.0,36.0
2,5.0,50.0,55.0,2.0
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [3]:
print(df.iloc[-1])

Unix            1502928000000
Date               2017-08-17
Symbol                ETHUSDT
Open                   301.13
High                   312.18
Low                     298.0
Close                   302.0
Volume ETH         7030.71034
Volume USDT    2154655.300357
tradecount               4522
Name: 2050, dtype: object


In [4]:
#NOTE: , in the end convert to a function, with filepath and interval as parameters
#read value
df= pd.read_csv('./historical_data/Binance_ETHUSDT_d.csv',header=1) # second row is column names
interval = '1d'
print(df)
print(df.dtypes)




               Unix        Date   Symbol     Open     High      Low    Close  \
0     1680048000000  2023-03-29  ETHUSDT  1773.00  1827.99  1772.48  1793.07   
1     1679961600000  2023-03-28  ETHUSDT  1715.21  1795.73  1701.13  1772.99   
2     1679875200000  2023-03-27  ETHUSDT  1773.88  1780.48  1686.13  1715.22   
3     1679788800000  2023-03-26  ETHUSDT  1741.88  1801.00  1739.31  1773.88   
4     1679702400000  2023-03-25  ETHUSDT  1749.57  1763.23  1713.09  1741.88   
...             ...         ...      ...      ...      ...      ...      ...   
2046  1503273600000  2017-08-21  ETHUSDT   299.10   346.52   294.60   323.29   
2047  1503187200000  2017-08-20  ETHUSDT   289.41   300.53   282.85   299.10   
2048  1503100800000  2017-08-19  ETHUSDT   293.31   299.90   278.00   290.91   
2049  1503014400000  2017-08-18  ETHUSDT   302.00   311.79   283.94   293.96   
2050  1502928000000  2017-08-17  ETHUSDT   301.13   312.18   298.00   302.00   

        Volume ETH   Volume USDT  trade

In [65]:
#TRANSFORMATION

print(df.columns)
#initial naming columns
col_list =[]
if(interval =='1d'):
    col_list = ['datetime','start_datetime','symbol','open','high','low','close','base_volume','quote_volume','num_trades']
df.columns = col_list

#NOTE in case of this dataset we need to add end_date and interval
df["interval"] = interval
if(interval =='1d'):
    df["close_datetime"] = pd.to_datetime(df['start_datetime'])+ pd.Timedelta(days=1)
print(df.shape)
#drop column if value null
df.dropna(subset=['symbol','start_datetime','close_datetime','open','close','high','low','base_volume'], inplace=True)
print(df.shape)

Index(['Unix', 'Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'Volume ETH',
       'Volume USDT', 'tradecount'],
      dtype='object')
(2051, 12)
(2051, 12)


In [66]:
display(df)

,datetime,start_datetime,symbol,open,high,low,close,base_volume,quote_volume,num_trades,interval,close_datetime
0,1680048000000,2023-03-29,ETHUSDT,1773.00,1827.99,1772.48,1793.07,536865.59720,9.673423e+08,810500,1d,2023-03-30
1,1679961600000,2023-03-28,ETHUSDT,1715.21,1795.73,1701.13,1772.99,536511.39730,9.355074e+08,766887,1d,2023-03-29
2,1679875200000,2023-03-27,ETHUSDT,1773.88,1780.48,1686.13,1715.22,570563.50230,9.871980e+08,810253,1d,2023-03-28
3,1679788800000,2023-03-26,ETHUSDT,1741.88,1801.00,1739.31,1773.88,354933.23720,6.276605e+08,592618,1d,2023-03-27
4,1679702400000,2023-03-25,ETHUSDT,1749.57,1763.23,1713.09,1741.88,355679.37430,6.203420e+08,620651,1d,2023-03-26
...,...,...,...,...,...,...,...,...,...,...,...,...
2046,1503273600000,2017-08-21,ETHUSDT,299.10,346.52,294.60,323.29,5219.44542,1.689472e+06,3925,1d,2017-08-22
2047,1503187200000,2017-08-20,ETHUSDT,289.41,300.53,282.85,299.10,2510.13871,7.428479e+05,2038,1d,2017-08-21
2048,1503100800000,2017-08-19,ETHUSDT,293.31,299.90,278.00,290.91,2146.19773,6.200226e+05,1795,1d,2017-08-20
2049,1503014400000,2017-08-18,ETHUSDT,302.00,311.79,283.94,293.96,9537.84646,2.858947e+06,5658,1d,2017-08-19
